In [59]:
#!pip install pymysql
#!pip install sqlalchemy
#!pip install mysqlclient
#!pip install cryptography


In [60]:
import pymysql
from bs4 import BeautifulSoup
import urllib.request
from urllib.parse import quote
import pandas as pd  

In [61]:
import MySQLdb

conn = MySQLdb.connect(
    user="mysqluser",
    passwd="mysqlpw",
    host="localhost",
    db="SCRAP",
    #charset="utf-8"
)
print(type(conn))
# <class 'MySQLdb.connections.Connection'>
cursor = conn.cursor()
print(type(cursor))
# <class 'MySQLdb.cursors.Cursor'>

<class 'MySQLdb.connections.Connection'>
<class 'MySQLdb.cursors.Cursor'>


In [68]:
def get_movie_reviews(mcode, page_num):
  movie_review_df = pd.DataFrame(columns =("Title", "Score", "Review"))
  url = "https://movie.naver.com/movie/point/af/list.naver?st=mcode&sword=" + str(mcode)+"&target=after"
  idx = 0 
  for _ in range(0, page_num):
    movie_page = urllib.request.urlopen(url).read()
    movie_page_soup = BeautifulSoup(movie_page, "html.parser")

    review_list = movie_page_soup.find_all("td", {"class": "title"})
    for review in review_list:
      title = review.find("a" , {"class": "movie color_b"}).get_text()
      score = review.find("em").get_text()
      review_text = review.find("a", {"class": "report"}).get("onclick").split(",")[2]
      movie_review_df.loc[idx] = [title,score,review_text]
      #TITLE = movie_review_df.loc[idx][0]
      
      #SCORE = movie_review_df.loc[idx][1]
      #REVIEW_TEXT = movie_review_df.loc[idx][2]
      #print(str(REVIEW_TEXT))
  
      cursor.execute(f"INSERT INTO CRAW_TABLE VALUES(\"{title}\",\"{score}\",\"{review_text}\")")
      conn.commit()
      idx+=1
      

      print("#",end="")
    try:
       url ="https://movie.naver.com" + movie_page_soup.find("a", {"class": "pg_next"}).get("href")
    except:
      break
  return movie_review_df

In [70]:
movie_review_df = get_movie_reviews(193855, 3)

movie_review_df
conn.commit()

##############################

In [ ]:
#cursor.execute(cursor.execute(f"INSERT INTO CRAWLING_TABLE(Title, Score, Review) VALUES(TITLE, SCORE, REVIEW))
#conn.commit()

In [ ]:
#alter database DB_NAME default character set utf8 collate utf8_general_ci;